In [220]:
import numpy as np
import pandas as pd
import sys
#!{sys.executable} -m pip install plotly
import plotly.graph_objects as go
from scipy.optimize import newton
from scipy.interpolate import interp1d
import numpy_financial as npf

Load Data for the selected 11 Bonds

In [221]:
data = pd.read_csv("10bonds_data.csv")
data

,Issuer,Coupon,ISIN,Issue Date,Maturity Date,Last Coupon Date,2025-01-06,2025-01-07,2025-01-08,2025-01-09,2025-01-10,2025-01-13,2025-01-14,2025-01-15,2025-01-16,2025-01-17,Bond URL
0,"Canada, Government of...",1.25%,CA135087K528,10/11/2019,3/1/2025,9/1/2024,99.73000,99.73000,99.74000,99.73000,99.74000,99.73000,99.73000,99.77000,99.78000,99.80000,https://markets.businessinsider.com/bonds/canadacd-bonds_201925-Bond-2025-ca135087k528
1,"Canada, Government of...",0.50%,CA135087K940,4/3/2020,9/1/2025,9/1/2024,98.40000,98.41000,98.40000,98.42000,98.37000,98.36000,98.36000,98.40000,98.47000,98.50000,https://markets.businessinsider.com/bonds/canadacd-bonds_202025-Bond-2025-ca135087k940
2,"Canada, Government of...",0.25%,CA135087L518,10/9/2020,3/1/2026,9/1/2024,96.99000,96.98000,96.97000,97.00000,96.90000,96.91000,96.80000,96.90000,97.01000,97.06000,https://markets.businessinsider.com/bonds/canadacd-bonds_202026-Bond-2026-ca135087l518
3,"Canada, Government of...",1.00%,CA135087L930,4/16/2021,9/1/2026,9/1/2024,97.01000,96.99000,97.00000,97.03000,96.86000,96.77000,96.71000,96.85000,97.03000,97.07000,https://markets.businessinsider.com/bonds/canadacd-bonds_202126-Bond-2026-ca135087l930
4,"Canada, Government of...",1.25%,CA135087M847,10/15/2021,3/1/2027,9/1/2024,96.62000,96.59000,96.58000,96.63000,96.39000,96.28000,96.19000,96.36000,96.60000,96.64000,https://markets.businessinsider.com/bonds/canadacd-bonds_202127-Bond-2027-ca135087m847
5,"Canada, Government of...",2.75%,CA135087N837,5/13/2022,9/1/2027,9/1/2024,99.63000,99.56000,99.54000,99.58000,99.27000,99.13000,99.02000,99.20000,99.50000,99.53000,https://markets.businessinsider.com/bonds/canadacd-bonds_202227-Bond-2027-ca135087n837
6,"Canada, Government of...",3.50%,CA135087P576,10/21/2022,3/1/2028,9/1/2024,101.82000,101.82000,101.70000,101.72000,101.52000,101.26000,101.02000,101.24000,101.58000,101.65000,https://markets.businessinsider.com/bonds/canadacd-bonds_202228-Bond-2028-ca135087p576
7,"Canada, Government of...",3.25%,CA135087Q491,4/21/2023,9/1/2028,9/1/2024,101.10000,101.14000,100.99000,100.94000,100.79000,100.49000,100.19000,100.42000,100.92000,101.02000,https://markets.businessinsider.com/bonds/canadacd-bonds_202328-Bond-2028-ca135087q491
8,"Canada, Government of...",4.00%,CA135087Q988,10/13/2023,3/1/2029,9/1/2024,104.07000,104.01000,103.90000,103.87000,103.53000,103.29000,102.99000,103.24000,103.80000,103.90000,https://markets.businessinsider.com/bonds/canadacd-bonds_202329-Bond-2029-ca135087q988
9,"Canada, Government of...",3.50%,CA135087R895,4/8/2024,9/1/2029,9/1/2024,102.22000,102.14000,102.04000,101.98000,101.59000,101.28000,101.00000,101.27000,101.85000,101.97000,https://markets.businessinsider.com/bonds/canadacd-bonds_202429-Bond-2029-ca135087r895


Suggested Naming Convention

In [222]:
name = data.copy()
name["coupon_rate"] = data["Coupon"].str.rstrip('%').astype(float).astype(str)

from datetime import datetime
name["Maturity"] = pd.to_datetime(name["Maturity Date"], format="%m/%d/%Y").dt.strftime("%b %y")
name["Name"] = "CAN " + name["coupon_rate"] + " " + name["Maturity"]

bonds_11 = pd.DataFrame({
    "ISIN": name["ISIN"],
    "Name": name["Name"]
})
bonds_11


,ISIN,Name
0,CA135087K528,CAN 1.25 Mar 25
1,CA135087K940,CAN 0.5 Sep 25
2,CA135087L518,CAN 0.25 Mar 26
3,CA135087L930,CAN 1.0 Sep 26
4,CA135087M847,CAN 1.25 Mar 27
5,CA135087N837,CAN 2.75 Sep 27
6,CA135087P576,CAN 3.5 Mar 28
7,CA135087Q491,CAN 3.25 Sep 28
8,CA135087Q988,CAN 4.0 Mar 29
9,CA135087R895,CAN 3.5 Sep 29


Covert Clean Price to Dirty Price

In [223]:
dirty = data[["ISIN", "Issue Date", "Maturity Date", "Last Coupon Date", "Coupon", "Bond URL"]].copy()

date_columns = ["2025-01-06", "2025-01-07", "2025-01-08", "2025-01-09", "2025-01-10",
                "2025-01-13", "2025-01-14", "2025-01-15", "2025-01-16", "2025-01-17"]

data["Last Coupon Date"] = pd.to_datetime(data["Last Coupon Date"])

data["Coupon"] = data["Coupon"].str.rstrip('%').astype(float) / 100

for date in date_columns:
    clean_price = data[date]
    days_difference = (pd.to_datetime(date) - data["Last Coupon Date"]).dt.days / 365
    dirty[date] = days_difference * data["Coupon"] * 100 + clean_price

dirty["Coupon"] = dirty["Coupon"].str.rstrip('%').astype(float) / 100

dirty

,ISIN,Issue Date,Maturity Date,Last Coupon Date,Coupon,Bond URL,2025-01-06,2025-01-07,2025-01-08,2025-01-09,2025-01-10,2025-01-13,2025-01-14,2025-01-15,2025-01-16,2025-01-17
0,CA135087K528,10/11/2019,3/1/2025,9/1/2024,0.01250,https://markets.businessinsider.com/bonds/canadacd-bonds_201925-Bond-2025-ca135087k528,100.16493,100.16836,100.18178,100.17521,100.18863,100.18890,100.19233,100.23575,100.24918,100.27260
1,CA135087K940,4/3/2020,9/1/2025,9/1/2024,0.00500,https://markets.businessinsider.com/bonds/canadacd-bonds_202025-Bond-2025-ca135087k940,98.57397,98.58534,98.57671,98.59808,98.54945,98.54356,98.54493,98.58630,98.65767,98.68904
2,CA135087L518,10/9/2020,3/1/2026,9/1/2024,0.00250,https://markets.businessinsider.com/bonds/canadacd-bonds_202026-Bond-2026-ca135087l518,97.07699,97.06767,97.05836,97.08904,96.98973,97.00178,96.89247,96.99315,97.10384,97.15452
3,CA135087L930,4/16/2021,9/1/2026,9/1/2024,0.01000,https://markets.businessinsider.com/bonds/canadacd-bonds_202126-Bond-2026-ca135087l930,97.35795,97.34068,97.35342,97.38616,97.21890,97.13712,97.07986,97.22260,97.40534,97.44808
4,CA135087M847,10/15/2021,3/1/2027,9/1/2024,0.01250,https://markets.businessinsider.com/bonds/canadacd-bonds_202127-Bond-2027-ca135087m847,97.05493,97.02836,97.02178,97.07521,96.83863,96.73890,96.65233,96.82575,97.06918,97.11260
5,CA135087N837,5/13/2022,9/1/2027,9/1/2024,0.02750,https://markets.businessinsider.com/bonds/canadacd-bonds_202227-Bond-2027-ca135087n837,100.58685,100.52438,100.51192,100.55945,100.25699,100.13959,100.03712,100.22466,100.53219,100.56973
6,CA135087P576,10/21/2022,3/1/2028,9/1/2024,0.03500,https://markets.businessinsider.com/bonds/canadacd-bonds_202228-Bond-2028-ca135087p576,103.03781,103.04740,102.93699,102.96658,102.77616,102.54493,102.31452,102.54411,102.89370,102.97329
7,CA135087Q491,4/21/2023,9/1/2028,9/1/2024,0.03250,https://markets.businessinsider.com/bonds/canadacd-bonds_202328-Bond-2028-ca135087q491,102.23082,102.27973,102.13863,102.09753,101.95644,101.68315,101.39205,101.63096,102.13986,102.24877
8,CA135087Q988,10/13/2023,3/1/2029,9/1/2024,0.04000,https://markets.businessinsider.com/bonds/canadacd-bonds_202329-Bond-2029-ca135087q988,105.46178,105.41274,105.31370,105.29466,104.96562,104.75849,104.46945,104.73041,105.30137,105.41233
9,CA135087R895,4/8/2024,9/1/2029,9/1/2024,0.03500,https://markets.businessinsider.com/bonds/canadacd-bonds_202429-Bond-2029-ca135087r895,103.43781,103.36740,103.27699,103.22658,102.84616,102.56493,102.29452,102.57411,103.16370,103.29329


4a - Yield

In [224]:
def calculate_ytm(bonds):
    ytms = []
    for i, (price, coupon_rate, maturity) in enumerate(sorted(bonds, key=lambda x: x[2])):
            time_periods = []
            current_time = maturity
            while current_time > 0:
                time_periods.append(current_time)
                current_time -= 0.5
            time_periods = time_periods[::-1]

            nper = len(time_periods)
            pmt = coupon_rate / 2
            fv = 100
            pv = -price

            ytm = npf.rate(nper=nper, pmt=pmt, pv=pv, fv=fv) * 2

            ytms.append(ytm)
    return ytms

In [225]:
fig = go.Figure()

for date in date_columns:
    bonds = []
    d = pd.to_datetime(date)

    Remaining_Maturity = (pd.to_datetime(dirty["Maturity Date"]) - d).dt.days / 365

    for i in range(len(data)):
        row = dirty.iloc[i]
        maturity = Remaining_Maturity.iloc[i]  # Get the maturity for this bond
        bond = (row[date], row["Coupon"] * 100, maturity)  # Price, Coupon, Maturity
        bonds.append(bond)

    bonds = [(float(price), float(coupon), float(maturity)) for price, coupon, maturity in bonds]

    yield_rates = calculate_ytm(bonds)
    maturities = [bond[2] for bond in bonds]

    # Add spot rates to figure
    fig.add_trace(
        go.Scatter(
            x=maturities,
            y=[rate * 100 for rate in yield_rates],  # Convert spot rates to percentages
            mode='lines+markers',
            name=str(date)
        )
    )

fig.update_layout(
    title='5-Year Yield Curve',
    xaxis_title='Maturity (Years)',
    yaxis_title='YTM (%)',
    legend_title="Dates"
)
fig.write_image("yield_curve.jpg")

fig.show()


4b - Spot Rate

In [226]:
def bootstrap_spot_curve(bonds):
    spot_rates = np.zeros(len(bonds))

    for i, (price, coupon_rate, maturity) in enumerate(sorted(bonds, key=lambda x: x[2])):

        # Generate time periods for semi-annual payments
        time_periods = []
        current_time = maturity
        while current_time > 0:
            time_periods.append(current_time)
            current_time -= 0.5
        time_periods = time_periods[::-1]

        # Generate cash flows for semi-annual payments
        num_payments = len(time_periods)  # Semi-annual payments
        cash_flows = np.array([coupon_rate / 2] * (num_payments - 1) + [100 + coupon_rate / 2])

        if i == 0:
            # First bond: Solve directly for the spot rate
            spot_rate = ((cash_flows[-1] / price)**(1 / maturity)) - 1
        else:
            # Subsequent bonds: Use existing spot rates to discount previous cash flows
            discounted_sum = sum(cash_flows[j] / (1 + spot_rates[j])**time_periods[j] for j in range(len(cash_flows) - 1))
            residual = price - discounted_sum
            spot_rate = ((cash_flows[-1] / residual)**(1 / maturity)) - 1

        spot_rates[i] = spot_rate

    return spot_rates

In [227]:
fig = go.Figure()

for date in date_columns:
    bonds = []
    d = pd.to_datetime(date)

    # Calculate remaining maturity for all rows
    Remaining_Maturity = (pd.to_datetime(dirty["Maturity Date"]) - d).dt.days / 365

    for i in range(len(data)):
        row = dirty.iloc[i]
        maturity = Remaining_Maturity.iloc[i]  # Get the maturity for this bond
        bond = (row[date], row["Coupon"] * 100, maturity)  # Price, Coupon, Maturity
        bonds.append(bond)

    # Convert bond data into float format
    bonds = [(float(price), float(coupon), float(maturity)) for price, coupon, maturity in bonds]

    # Bootstrap spot rates
    spot_rates = bootstrap_spot_curve(bonds)
    maturities = [bond[2] for bond in bonds]

    # Add spot rates to figure
    fig.add_trace(
        go.Scatter(
            x=maturities,
            y=[rate * 100 for rate in spot_rates],  # Convert spot rates to percentages
            mode='lines+markers',
            name=str(date)
        )
    )

fig.update_layout(
    title='Spot Curve (1-5 Years)',
    xaxis_title='Maturity (Years)',
    yaxis_title='Spot Rate (%)',
    legend_title="Dates"
)

fig.show()


4c - Forward Rate

In [228]:
def calculate_1y_forward_curve_semiannual(spot_rates, maturities):
    forward_curve = []  # List to store calculated forward rates

    for i in range(1, len(spot_rates)):
        t = maturities[0]
        t_plus_n = maturities[i]
        n = t_plus_n - t
        S_t = spot_rates[0]
        S_t_plus_n = spot_rates[i]

        forward_rate = (
            ( (1 + S_t_plus_n) ** (2 * t_plus_n) / (1 + S_t) ** (2 * t) ) ** (1 / (2*n)) - 1
        )
        forward_curve.append(forward_rate)

    return forward_curve


In [229]:
fig = go.Figure()

for date in date_columns:
    bonds = []
    d = pd.to_datetime(date)

    # Calculate remaining maturity for all rows
    Remaining_Maturity = (pd.to_datetime(dirty["Maturity Date"]) - d).dt.days / 365

    for i in range(len(data)):
        row = dirty.iloc[i]
        maturity = Remaining_Maturity.iloc[i]  # Get the maturity for this bond
        bond = (row[date], row["Coupon"] * 100, maturity)  # Price, Coupon, Maturity
        bonds.append(bond)

    bonds = [
        (float(price), float(coupon), float(maturity))
        for price, coupon, maturity in bonds
    ]

    spot_rates = bootstrap_spot_curve(bonds)
    maturities = [bond[2] for bond in bonds]
    spot_rates = spot_rates[2:][::2]
    maturities = maturities[2:][::2]
    forward_rates = calculate_1y_forward_curve_semiannual(spot_rates, maturities)
    maturities = maturities[1:]

    fig.add_trace(
        go.Scatter(
            x=maturities,
            y=[rate * 100 for rate in forward_rates],
            mode='lines+markers',
            name=str(date)
        )
    )

fig.update_layout(
    title='Forward Curve (2-5 Years)',
    xaxis_title='Maturity (Years)',
    yaxis_title='Forward Rate (%)',
    legend_title="Dates"
)

fig.show()


5 - Covariance Matrix

In [230]:
ytm_matrix = pd.DataFrame()
forward_matrix = pd.DataFrame()
for date in date_columns:
    bonds = []
    d = pd.to_datetime(date)

    # Calculate remaining maturity for all rows
    Remaining_Maturity = (pd.to_datetime(dirty["Maturity Date"]) - d).dt.days / 365

    for i in range(len(data)):
        row = dirty.iloc[i]
        maturity = Remaining_Maturity.iloc[i]  # Get the maturity for this bond
        bond = (row[date], row["Coupon"] * 100, maturity)  # Price, Coupon, Maturity
        bonds.append(bond)

    # Convert bond data into float format
    bonds = [(float(price), float(coupon), float(maturity)) for price, coupon, maturity in bonds]

    # Bootstrap spot rates
    yield_rates = calculate_ytm(bonds)
    ytm_matrix[date] = yield_rates

    spot_rates = bootstrap_spot_curve(bonds)
    maturities = [bond[2] for bond in bonds]
    spot_rates = spot_rates[2:][::2]
    maturities = maturities[2:][::2]
    forward_rates = calculate_1y_forward_curve_semiannual(spot_rates, maturities)
    forward_matrix[date] = forward_rates

In [231]:
ytm_matrix

,2025-01-06,2025-01-07,2025-01-08,2025-01-09,2025-01-10,2025-01-13,2025-01-14,2025-01-15,2025-01-16,2025-01-17
0,0.00919,0.00912,0.00885,0.00898,0.00871,0.00871,0.00864,0.00777,0.00750,0.00703
1,0.01947,0.01935,0.01944,0.01922,0.01972,0.01978,0.01977,0.01934,0.01861,0.01829
2,0.02243,0.02249,0.02255,0.02234,0.02303,0.02295,0.02371,0.02301,0.02224,0.02189
3,0.02360,0.02369,0.02363,0.02345,0.02433,0.02476,0.02506,0.02431,0.02335,0.02313
4,0.02472,0.02483,0.02486,0.02464,0.02564,0.02606,0.02643,0.02569,0.02466,0.02448
5,0.02546,0.02567,0.02572,0.02555,0.02660,0.02701,0.02737,0.02672,0.02565,0.02552
6,0.02587,0.02584,0.02616,0.02608,0.02664,0.02733,0.02801,0.02733,0.02629,0.02606
7,0.02658,0.02646,0.02683,0.02693,0.02730,0.02802,0.02879,0.02816,0.02682,0.02654
8,0.02703,0.02714,0.02737,0.02741,0.02817,0.02865,0.02933,0.02872,0.02740,0.02714
9,0.02759,0.02774,0.02793,0.02804,0.02885,0.02945,0.03002,0.02943,0.02817,0.02790


In [232]:
forward_matrix

,2025-01-06,2025-01-07,2025-01-08,2025-01-09,2025-01-10,2025-01-13,2025-01-14,2025-01-15,2025-01-16,2025-01-17
0,0.02848,0.02867,0.02864,0.02839,0.02989,0.03110,0.03087,0.03007,0.02862,0.02870
1,0.02886,0.02875,0.02931,0.02933,0.02978,0.03110,0.03177,0.03109,0.02982,0.02967
2,0.02984,0.02999,0.03031,0.03051,0.03132,0.03210,0.03276,0.03220,0.03054,0.03033
3,0.03049,0.03106,0.03120,0.03124,0.03261,0.03357,0.03389,0.03306,0.03154,0.03147


Covariance Matrix of the Daily log-returns of Yield:

In [233]:
ytm_matrix = ytm_matrix.iloc[2:].reset_index(drop=True)
ytm_rows = ytm_matrix.iloc[::2].values
ytm_log_returns = np.log(ytm_rows.T[1:]/ytm_rows.T[:-1]).T
ytm_cov = np.cov(ytm_log_returns)
ytm_cov

array([[0.00054531, 0.00050576, 0.00045659, 0.00049263, 0.00050787],
       [0.00050576, 0.00058385, 0.00050105, 0.00053704, 0.00059917],
       [0.00045659, 0.00050105, 0.00050839, 0.00051281, 0.00052153],
       [0.00049263, 0.00053704, 0.00051281, 0.00055038, 0.00056633],
       [0.00050787, 0.00059917, 0.00052153, 0.00056633, 0.00063106]])

Covariance Matrix of the Daily log-returns of Forward Rates:

In [234]:
forward_rows = forward_matrix.values
forward_log_returns = np.log(forward_rows.T[1:]/forward_rows.T[:-1]).T
forward_cov = np.cov(forward_log_returns)
forward_cov

array([[0.00094262, 0.00061051, 0.00064834, 0.00079544],
       [0.00061051, 0.00063615, 0.00058021, 0.00056898],
       [0.00064834, 0.00058021, 0.00062542, 0.00063477],
       [0.00079544, 0.00056898, 0.00063477, 0.00073688]])

6 - Eigenvalues & Eigenvectors

In [235]:
yield_eigenvalues, yield_eigenvectors = np.linalg.eig(ytm_cov)

print("Eigenvalues for Covariance Matrix of Yield:")
for i, eigenvalue in enumerate(yield_eigenvalues, 1):
    print(f"  {i}: {eigenvalue:.8e}")

print("\nEigenvectors for Covariance Matrix of Yield:")
for i, eigenvector in enumerate(yield_eigenvectors.T, 1):  # Transpose for easier labeling
    print(f"  {i}: {eigenvector}")


Eigenvalues for Covariance Matrix of Yield:
  1: 2.65027180e-03
  2: 8.98523602e-05
  3: 6.10333908e-05
  4: 2.91631527e-06
  5: 1.49121178e-05

Eigenvectors for Covariance Matrix of Yield:
  1: [-0.42284294 -0.46120231 -0.42200135 -0.44916832 -0.47818298]
  2: [-0.88702967  0.17310665  0.14765991  0.10601972  0.38751623]
  3: [-0.10621205 -0.46710372  0.70723308  0.33683567 -0.39610128]
  4: [-0.1517174   0.57270906 -0.21259665  0.43788404 -0.6419084 ]
  5: [ 0.00911311 -0.45950513 -0.50471323  0.69412315  0.22853794]


In [236]:
forward_eigenvalues, forward_eigenvectors = np.linalg.eig(forward_cov)

print("Eigenvalues for Covariance Matrix of Forward Rates:")
for i, eigenvalue in enumerate(forward_eigenvalues, 1):
    print(f"  {i}: {eigenvalue:.8e}")

print("\nEigenvectors for Covariance Matrix of Forward Rates:")
for i, eigenvector in enumerate(forward_eigenvectors.T, 1):  # Transpose for easier labeling
    print(f"  {i}: {eigenvector}")


Eigenvalues for Covariance Matrix of Forward Rates:
  1: 2.67653840e-03
  2: 1.94532237e-04
  3: 6.13779176e-05
  4: 8.61563016e-06

Eigenvectors for Covariance Matrix of Forward Rates:
  1: [0.56659934 0.44517227 0.46437906 0.51491642]
  2: [ 0.60240071 -0.6690951  -0.36181145  0.24190412]
  3: [ 0.43614802  0.54330815 -0.56987358 -0.43570089]
  4: [ 0.35475834 -0.24279548  0.57331376 -0.6975014 ]
